# Sort Data 

## Imports

In [52]:
import pandas as pd
import numpy as np
import json

from geopy.distance import geodesic

## Functions

In [53]:
def census_diff(df, census):

    grp = df.groupby(["plot",  census])
    cen = grp.date.agg(['min', 'max'])
    cen["mid"] = (cen["min"] + (cen["max"] - cen["min"])/2).dt.date
    cen["difference"] = cen["mid"].diff().astype('timedelta64[D]')
    cen.loc[cen["difference"] < 0 , "difference"] = np.NAN
    cen["diff_yrs"] = cen.difference/365

    cen.reset_index(level=0, inplace=True)
    cen.reset_index(level=0, inplace=True)

    cen[census].astype(str)
    cen["step"] = cen[census].astype(str).shift() + "-" + cen[census].astype(str)
    cen.loc[cen["difference"].isnull(), "step"] = np.NaN
    cen.index = cen['plot'] + "_" + cen['step']

    return cen

## Open general data

In [54]:
# open plot locations
with open('../Data/rows.geojson') as f:
    data = json.load(f)

properties = pd.DataFrame()

# add each line of geojson file to dataframe
for i in range(0, len(data['features'])):
    properties = properties.append(pd.DataFrame(data['features'][i]['properties'], index = [i]))

# only really care about these columns
properties = properties.loc[: , ["plot_size",
                                 "centroid_y",
                                 "centroid_x",
                                 "fractal_order",
                                 "location"]]

properties.rename(columns={'centroid_y':'longitude', 'centroid_x':'latitude'}, inplace=True)
properties["longlat"] = properties.apply(lambda x: [x.longitude, x.latitude], axis=1)

# seperate dataframe for only second order fractal points
second_order = properties.loc[properties.fractal_order == 2, : ]

# function to find point closest to given point
def closest(pt, others):
    
    clst_pt = min(others.longlat, key = lambda x: geodesic(pt, x).meters)
        
    return others.location.loc[others.longlat.apply(lambda x: x == clst_pt)].reset_index(drop = True)


# fractal nesting and agb data
fpn = pd.read_csv("../Data/Fractal_point_nesting.csv")
agb = pd.read_csv("../Data/AGB.csv")

# specific wanted columns - and rename ***(going with Chave moist)***
agb = agb[["field_name", "Plot", "Date", "AGB_Chave_moist", "ForestQuality"]]
agb.columns = ["field_name", "plot", "date", "agb", "forestquality"]

## Mammals Data

#### Open and sort the data

In [55]:
# open each seperate plot
E  = pd.read_csv("../Data/small_mammals/test/E_test.csv")
F  = pd.read_csv("../Data/small_mammals/test/F_test.csv")
D  = pd.read_csv("../Data/small_mammals/test/D_test.csv")
OG = pd.read_csv("../Data/small_mammals/test/OG_test.csv")

#### function to sort out each plot in turn

In [56]:
def sort_mams(df):

    # new column names
    ncolnames   = ["occasion", "date", "grid", "point", "trap", "trap_id", "species"]
    df.columns  = ncolnames

    # stupid formatting sorted
    df["occasion"]  = df.occasion.str.replace("--", "-")
    df["grid"]      = df.grid.str.replace("--", "-")
    df["trap_id"]   = df.trap_id.str.replace("--", "-")
    df["trap_id"]   = df.trap_id.apply(lambda x: x[:-1])
    df["date"]      = pd.to_datetime(df.date)
    df["year"]      = df.date.dt.year
    df["census"]    = df.occasion.str.partition("-")[2].str.partition("-")[2]
    df["plot"]      = df.occasion.str[0]
    
    return df

E  = sort_mams(E)
F  = sort_mams(F)
D  = sort_mams(D)
OG = sort_mams(OG)

frames = [E, F, D, OG]

mamls_df = pd.concat(frames, sort = False)

mamls_df["species"] = mamls_df.species.fillna("None")

#### match with species names from lookup

In [57]:
# mammals species lookup table 
m_lkup = pd.read_csv("../Data/small_mammals/mammals_lookup.csv")
m_lkup.columns = ["code", "species", "scientific"]

mamls_df["species"] = mamls_df.species.str.strip()


# my fairly questionable decisions...

# if its a questionmark - I just go with it
# if its an either or I go with the first one!
mamls_df.loc[mamls_df.loc[:, "species"] == "CTRS-but see notes", "species"] = "CTRS"

mamls_df.loc[mamls_df.loc[:, "species"] == "SS?",          "species"] = "SS"
mamls_df.loc[mamls_df.loc[:, "species"] == "WH?",          "species"] = "WH"
mamls_df.loc[mamls_df.loc[:, "species"] == "PR?",          "species"] = "PR"
mamls_df.loc[mamls_df.loc[:, "species"] == "RR?",          "species"] = "RR"
mamls_df.loc[mamls_df.loc[:, "species"] == "MR?",          "species"] = "MR"
mamls_df.loc[mamls_df.loc[:, "species"] == "MR??",         "species"] = "MR"
mamls_df.loc[mamls_df.loc[:, "species"] == "RS?",          "species"] = "RS"
mamls_df.loc[mamls_df.loc[:, "species"] == "LGTRS?",       "species"] = "LGTRS"
mamls_df.loc[mamls_df.loc[:, "species"] == "BS?",          "species"] = "BS"
mamls_df.loc[mamls_df.loc[:, "species"] == "PSQ",          "species"] = "LSQ"      # not confident on this
mamls_df.loc[mamls_df.loc[:, "species"] == "BSQ",          "species"] = "BSQ?"
mamls_df.loc[mamls_df.loc[:, "species"] == "SSQ",          "species"] = "SSQ?"
mamls_df.loc[mamls_df.loc[:, "species"] == "RS or SS" ,    "species"] = "RS"
mamls_df.loc[mamls_df.loc[:, "species"] == "WH or SS",     "species"] = "WH"
mamls_df.loc[mamls_df.loc[:, "species"] == "BS/RS?",       "species"] = "BS"
mamls_df.loc[mamls_df.loc[:, "species"] == "PTSQ?",        "species"] = "PTSQ"
mamls_df.loc[mamls_df.loc[:, "species"] == "LETRS",        "species"] = "LETRS?"   # for some reason the lookup table has a ?
mamls_df.loc[mamls_df.loc[:, "species"] == "CBS?",         "species"] = "CBS"
mamls_df.loc[mamls_df.loc[:, "species"] == "SL?TRS",       "species"] = "SLTRS"
mamls_df.loc[mamls_df.loc[:, "species"] == "SLTRS?",       "species"] = "SLTRS"
mamls_df.loc[mamls_df.loc[:, "species"] == "L?TRS",        "species"] = "SLTRS"
mamls_df.loc[mamls_df.loc[:, "species"] == "LSQ?",         "species"] = "LSQ"
mamls_df.loc[mamls_df.loc[:, "species"] == "CTRS?",        "species"] = "CTRS"
mamls_df.loc[mamls_df.loc[:, "species"] == "LTRS or CTRS", "species"] = "CTRS"     # went with CTRS as LTRS could refer to a couple
mamls_df.loc[mamls_df.loc[:, "species"] == "LTRS",         "species"] = "LETRS?"   # not convinced about this one
mamls_df.loc[mamls_df.loc[:, "species"] == "Squirrel",     "species"] = "squirrel"
mamls_df.loc[mamls_df.loc[:, "species"] == "DTT_DEAD",     "species"] = "DTT"
mamls_df.loc[mamls_df.loc[:, "species"] == "LSQ?_DEAD",    "species"] = "LSQ"
mamls_df.loc[mamls_df.loc[:, "species"] == "squirrel",     "species"] = "unknown"  # ***mmm?***
mamls_df.loc[mamls_df.loc[:, "species"] == "See notes",    "species"] = "unknown"  # i'm effectivley treating 'unknown' as a seperate species which seems spurious at best 
mamls_df.loc[mamls_df.loc[:, "species"] == "??",           "species"] = "unknown"
mamls_df.loc[mamls_df.loc[:, "species"] == "?",            "species"] = "unknown"
mamls_df.loc[mamls_df.loc[:, "species"] == "Unknown",      "species"] = "unknown"

# merge
mamls_df = pd.merge(mamls_df,
                    m_lkup[["code", "scientific"]],
                    how      = "left",
                    left_on  = "species",
                    right_on = "code")

# get rid of the leftovers... (there were a couple of birds/reptiles)
mamls_df = mamls_df.loc[-mamls_df.code.isna(), :]

#### find closest f2 point to each trap and get agb measure

In [58]:
# get all the unique trap names
trap_locs = pd.DataFrame({"trap_id" : mamls_df.trap_id.unique()})

trap_locs = trap_locs.merge(properties[["location", "longlat"]],
                            how      = "left",
                            left_on  = "trap_id",
                            right_on = "location")

# find the closest second order fractal point
trap_locs["second_order"] = trap_locs.longlat.apply(lambda x: closest(x, second_order))

# merge back to master dataframe
mamls_df = mamls_df.merge(trap_locs, how = "left", on = "trap_id")

# just want the point number 
mamls_df.second_order = mamls_df.second_order.str[-3:]
mamls_df.second_order = mamls_df.second_order.astype(int)

# merge to get agb and forest quality
mamls_df = mamls_df.merge(agb[["plot", "agb", "forestquality"]], how = "left",
                          left_on = "second_order", right_on = "plot")

#### final cleanup and save

In [59]:
mamls_df = mamls_df.rename(index=str, columns={"plot_x": "plot"})

mamls_df = mamls_df[["occasion",
                     "date",
                     "grid",
                     "point",
                     "trap",
                     "trap_id",
                     "species",
                     "year",
                     "plot",
                     "census",
                     "scientific",
                     "longlat",
                     "second_order",
                     "agb",
                     "forestquality"]]

mamls_df.to_csv("../Results/mammals_sorted2.csv")

#### Make the species/plot matrix

In [60]:
# i'll give four different combinations a go...
mamls_df["trap_year"]   = mamls_df["plot"] + "_" + mamls_df.trap_id + "_" + mamls_df.year.astype(str)
mamls_df["grid_year"]   = mamls_df["plot"] + "_" + mamls_df.grid    + "_" + mamls_df.year.astype(str)
mamls_df["trap_census"] = mamls_df["plot"] + "_" + mamls_df.trap_id + "_" + mamls_df.census
mamls_df["grid_census"] = mamls_df["plot"] + "_" + mamls_df.grid    + "_" + mamls_df.census

# function to make matrix
def make_matrix(df, what):
    mx = df.groupby([what, "species"]).size().unstack()  # groupby whatever i've chosen and species
    mx = mx.fillna(value = 0)                            # fill with 0's
    mx = mx.drop("None", axis = 1)                       # so we keep plot row even if nothing was trapped
    return mx

# do
mamls_TY = make_matrix(mamls_df, "trap_year")
mamls_GY = make_matrix(mamls_df, "grid_year")
mamls_TC = make_matrix(mamls_df, "trap_census")
mamls_GC = make_matrix(mamls_df, "grid_census")

# save
mamls_TY.to_csv("../Results/m_trap-year.csv")
mamls_GY.to_csv("../Results/m_grid-year.csv")
mamls_TC.to_csv("../Results/m_trap-census.csv")
mamls_GC.to_csv("../Results/m_grid_census.csv")

#### agb

In [101]:
# get the agb (either mean or median of all the traps in plot)
mamls_agb = mamls_df.groupby("plot").agb.describe()
mamls_agb = pd.DataFrame(mamls_agb["50%"])
mamls_agb.to_csv("../Results/mamls_agb.csv")

#### standardise time

In [82]:
# and time difference between cesuses - i'm doing year or occasion...
mamls_cn_diff = census_diff(mamls_df, "census")
mamls_yr_diff = census_diff(mamls_df, "year")

mamls_yr_diff = mamls_yr_diff.rename(index=str, columns={"year": "census"})

mamls_cn_diff.to_csv("../Results/mammals_census_dates.csv")
mamls_yr_diff.to_csv("../Results/mammals_years_dates.csv")

## Tree Data

#### readin raw data

In [ ]:
# readin RAW data
tree_df = pd.read_csv("../Data/SAFE_CarbonPlots_Tree+LianaCensus.csv")

#### function to sort everything out for each census

In [ ]:
def sort_data(df, census_no):  # give new column names, delete NAs and dead...

    # consistant and better column names
    new_Cnames = ['f_type',       # forest type
                  'plot',
                  'subplot',
                  'date',         # date of measurements
                  'observers',
                  'tag_no',
                  'd_pom',        # diameter of tree (cm)
                  'h_pom',        # height diameter is taken (m) 1.3 by default
                  'height',
                  'flag',         # condition of trees (see flag list)
                  'alive',        # 1 = yes, NaN = no
                  'stem_C',       # aboveground biomass of tree (kg)
                  'root_C',       # root biomass of tree
                  'field_cmnts',  # comments from field
                  'data_cmnts',   # comments from data entry
                  'sbplt_X',
                  'sbplt_Y',
                  'CPA',          # projected area of the crown of the stem
                  'X_FMC',        # plot level X coordinate
                  'Y_FMC',        # plot level Y coordinte
                  'Z_FMC',        # plot level elevation
                  'family',
                  'binomial',
                  'wood_density']

    # give each census these column names
    df.columns = new_Cnames

    # get unique ID - combine plot and tag_no
    df = df.assign(ID = df['plot'] + df['tag_no'].map(str))

    # column with census number
    df = df.assign(census = census_no)

    # delete rows with NaNs in important columns
    impt_cols = ['tag_no', 'd_pom', 'h_pom', 'height', 'flag', 'alive',
                 'stem_C', 'root_C']

    df = df.dropna(subset = impt_cols, how = 'all')

    # delete dead trees (alive == 0)
    df = df[df.alive == 1]

    # sort out dates
    df.date = pd.to_datetime(df.date, dayfirst = True)

    return df

#### subset each census - weird column things...

In [ ]:
# subset for each census
census_1 = tree_df.iloc[ :, list(range(0, 3))     # same for all
                          + list(range(3, 15))    # specific for census
                          + list(range(53, 62))]  # same for all

census_2 = tree_df.iloc[ :, list(range(0, 3))
                          + list(range(15, 27))
                          + list(range(53, 62))]

census_3 = tree_df.iloc[ :, list(range(0, 3))
                          + list(range(27, 39))
                          + list(range(53, 62))]

census_4 = tree_df.iloc[ :, list(range(0, 3))
                          + list(range(39, 51))
                          + list(range(53, 62))]

#### do function and combine census'

In [ ]:
# sort data for each census
census_1 = sort_data(census_1, 1)
census_2 = sort_data(census_2, 2)
census_3 = sort_data(census_3, 3)
census_4 = sort_data(census_4, 4)

# recombine all census data (stack on top of each other)
tree_df = pd.concat([census_1, census_2, census_3, census_4], ignore_index = True)

#### add extra columns and save

In [ ]:
# add genus column
tree_df['plot']        = tree_df['plot'].replace(" ", "", regex=True)
tree_df['subplot']     = tree_df['subplot'].apply(lambda x: str(x).zfill(2))
tree_df['genus']       = tree_df.apply(lambda row: row.binomial.split(" ")[0], axis = 1)
tree_df['plt_sub']     = tree_df['plot'] + "_sp" + tree_df['subplot'].astype(str)
tree_df['plt_sub_cen'] = tree_df['plt_sub'] + "_c" + tree_df['census'].astype(str)
tree_df['plot_c']      = tree_df['plot'] + "_c" + tree_df['census'].astype(str)
tree_df['census']      = "c" + tree_df.census.astype(str)

# save to csv
tree_df.to_csv("../Results/trees_sorted.csv", index = False)

#### make matrix

In [90]:
# species matrix
tree_matrix = tree_df.groupby(['plt_sub_cen', 'binomial']).size().unstack()
tree_matrix = tree_matrix.fillna(value = 0)
tree_matrix.to_csv("../Results/trees_matrix.csv")

tree_genus_matrix = tree_df.groupby(['plt_sub_cen', 'genus']).size().unstack()
tree_genus_matrix = tree_genus_matrix.fillna(value = 0)
tree_genus_matrix.to_csv("../Results/trees_genus_matrix.csv")

tree_family_matrix = tree_df.groupby(['plt_sub_cen', 'family']).size().unstack()
tree_family_matrix = tree_family_matrix.fillna(value = 0)
tree_family_matrix.to_csv("../Results/trees_family_matrix.csv")

#### trees agb (different from the others as calculated from data)

In [94]:
# total biomass at each census for each plot
trees_agb = pd.DataFrame(tree_df.groupby(["plot", "census"]).stem_C.sum())

# take the mean of all census' 0.0625 and 0.001 to get it into Mg/0.0625 ha
trees_agb = pd.DataFrame(trees_agb.groupby("plot").mean()*  0.0625 * 0.001)

trees_agb.to_csv("../Results/trees_agb.csv")

#### trees census standardise time

In [106]:
tree_cen = census_diff(tree_df, "census")
tree_cen.to_csv("../Results/trees_census_dates.csv")